In [58]:
import numpy as np, pandas as pd
import os,glob, re

dfs = {re.search('/([^/\.]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob(os.getcwd()+'/Raw/*.csv')}
print('data frames read:{}'.format(list(dfs.keys())))

print('local variables with the same names are created.')
for k, v in dfs.items(): locals()[k] = v

data frames read:['air_reserve', 'air_store_info', 'air_visit_data', 'date_info', 'hpg_reserve', 'hpg_store_info', 'sample_submission', 'store_id_relation']
local variables with the same names are created.


In [2]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [9]:
weekdayholidays = date_info.apply(lambda x: x.day_of_week in ['Saturday','Sunday'] and x.holiday_flg == 1,axis=1)
date_info.loc[weekdayholidays,'holiday_flg'] = 0
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,0
2,2016-01-03,Sunday,0
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [19]:
date_info['weights'] = (date_info.index + 1)/ len(date_info)
date_info.head()

,calendar_date,day_of_week,holiday_flg,weights
0,2016-01-01,Friday,1,0.001934
1,2016-01-02,Saturday,0,0.003868
2,2016-01-03,Sunday,0,0.005803
3,2016-01-04,Monday,0,0.007737
4,2016-01-05,Tuesday,0,0.009671


In [20]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [21]:
trainset = air_visit_data.merge(date_info,how= 'left', left_on = 'visit_date', right_on = 'calendar_date')
trainset.drop('calendar_date', inplace = True, axis = 1)

In [33]:
trainset['visitors'] = trainset['visitors'].apply(pd.np.log1p)
trainset.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weights
0,air_ba937bf13d40fb24,2016-01-13,3.258097,Wednesday,0,0.025145
1,air_ba937bf13d40fb24,2016-01-14,3.496508,Thursday,0,0.027079
2,air_ba937bf13d40fb24,2016-01-15,3.401197,Friday,0,0.029014
3,air_ba937bf13d40fb24,2016-01-16,3.135494,Saturday,0,0.030948
4,air_ba937bf13d40fb24,2016-01-18,1.945910,Monday,0,0.034816


In [34]:
weighted_mean = lambda x : ((x.visitors * x.weights).sum() / (x.weights).sum())
visitors_per_weekday = trainset.groupby(['air_store_id','day_of_week','holiday_flg']).apply(weighted_mean).reset_index()

In [35]:
visitors_per_weekday.rename(columns={0:'visitors'},inplace = True)
visitors_per_weekday.head()

,air_store_id,day_of_week,holiday_flg,visitors
0,air_00a91d42b08b08d9,Friday,0,3.590026
1,air_00a91d42b08b08d9,Monday,0,3.091672
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.526836
4,air_00a91d42b08b08d9,Sunday,0,1.098612
5,air_00a91d42b08b08d9,Thursday,0,3.414904
6,air_00a91d42b08b08d9,Tuesday,0,3.173611
7,air_00a91d42b08b08d9,Wednesday,0,3.330028
8,air_0164b9927d20bcc3,Friday,0,2.327400
9,air_0164b9927d20bcc3,Friday,1,2.747824


In [59]:
sample_submission["airstore"],sample_submission["date"] = sample_submission.id.str[:20],sample_submission.id.str[21:]
sample_submission.head()

,id,visitors,airstore,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [60]:
sample_submission = sample_submission.merge(date_info,how = 'left',left_on = 'date',right_on = 'calendar_date')
sample_submission.tail()

,id,visitors,airstore,date,calendar_date,day_of_week,holiday_flg
32014,air_fff68b929994bfbd_2017-05-27,0,air_fff68b929994bfbd,2017-05-27,2017-05-27,Saturday,0
32015,air_fff68b929994bfbd_2017-05-28,0,air_fff68b929994bfbd,2017-05-28,2017-05-28,Sunday,0
32016,air_fff68b929994bfbd_2017-05-29,0,air_fff68b929994bfbd,2017-05-29,2017-05-29,Monday,0
32017,air_fff68b929994bfbd_2017-05-30,0,air_fff68b929994bfbd,2017-05-30,2017-05-30,Tuesday,0
32018,air_fff68b929994bfbd_2017-05-31,0,air_fff68b929994bfbd,2017-05-31,2017-05-31,Wednesday,0


In [63]:
sample_submission.drop('visitors',inplace=True,axis = 1)
sample_submission = sample_submission.merge(visitors_per_weekday,how = "left",left_on = ['airstore','day_of_week','holiday_flg'], 
                                            right_on = ['air_store_id','day_of_week','holiday_flg'])
sample_submission.head()

,id,airstore,date,calendar_date,day_of_week,holiday_flg,air_store_id,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,2017-04-23,Sunday,0,air_00a91d42b08b08d9,1.098612
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,2017-04-24,Monday,0,air_00a91d42b08b08d9,3.091672
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,2017-04-25,Tuesday,0,air_00a91d42b08b08d9,3.173611
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,2017-04-26,Wednesday,0,air_00a91d42b08b08d9,3.330028
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,2017-04-27,Thursday,0,air_00a91d42b08b08d9,3.414904


In [72]:
sample_submission.apply(pd.notnull).all(axis=0)

id                True
airstore          True
date              True
calendar_date     True
day_of_week       True
holiday_flg       True
air_store_id     False
visitors         False
dtype: bool